In [257]:
import pandas as pd

In [160]:
dates = pd.read_gbq(
    query='select distinct date(deposit_initiated_ts) as date from transactions.fact_deposits',
    project_id='friktion-dev'
    )

In [161]:
dates.describe()

,date
count,99
unique,99
top,2021-12-26
freq,1


In [162]:
users = pd.read_gbq(
    query='select distinct(user_address) as user_address from transactions.fact_deposits')

In [163]:
users.describe()

,user_address
count,12426
unique,12426
top,FrrrZWDn7pCMiGoh35PVZbP8uV2n5nFFJx3uFuyKty2R
freq,1


In [164]:
user_first_deposits_dates = pd.read_gbq(
    query='select user_address, min(date(deposit_initiated_ts)) as first_deposit_dt from transactions.fact_deposits group by user_address')

In [165]:
user_first_deposits_dates.describe()

,user_address,first_deposit_dt
count,12426,12426
unique,12426,99
top,G3fVq24iJjqidFCqbmQqPWvfGvRWzqAyzZFH8iJPjEnP,2022-03-30
freq,1,2160


In [166]:
users.iloc[0][0]

'FrrrZWDn7pCMiGoh35PVZbP8uV2n5nFFJx3uFuyKty2R'

In [384]:
user_first_deposits_dates[user_first_deposits_dates['user_address'].str.contains('6o2ASdEoRGDZaaHRupBmocL7wKWNpyMXkFKePGsSs79p')]

,user_address,first_deposit_dt
7,6o2ASdEoRGDZaaHRupBmocL7wKWNpyMXkFKePGsSs79p,2021-12-28


In [73]:
# eventuall you'll iterate through this activity of making a user table per userAddress and then unioning together with all the others

In [171]:
user_first_deposits_dates.shape

(12426, 2)

In [387]:
# to make this iterate through all users increment the first value in the iloc call
# for index in xrange(user_first_deposits_dates.shape[0]):
index = 7
current_user_address = user_first_deposits_dates.iloc[index,:]['user_address']
user_first_deposit_date = user_first_deposits_dates.iloc[index,:]['first_deposit_dt']

In [388]:
user_deposit_table = pd.DataFrame(columns=['user_address', 'as_of_date', 'total_value_locked_USD', 'total_deposited_USD', 'total_withdrawn_USD', \
                                           'tvl_delta_30_days', 'tvl_delta_60_days', 'tvl_delta_90_days', \
                                           'tvl_delta_1_epoch', 'tvl_delta_2_epoch', \
                                           'tvl_delta_3_epoch', 'tvl_delta_4_epoch', 'tvl_delta5_epoch', \
                                           'first_deposit_date', 'first_deposit_epoch', 'first_deposit_amount', 'first_deposit_token', \
                                           'last_deposit_date', 'last_deposit_epoch', 'last_deposit_amount', 'last_deposit_token', \
                                           'days_since_last_deposit', 'epochs_since_last_deposit', \
                                           'first_withdrawal_date', 'first_withdrawal_epoch', 'first_withdrawal_amount', 'first_withdrawal_token', \
                                           'last_withdrawal_date', 'last_withdrawal_epoch', 'last_withdrawal_amount', 'last_withdrawal_token', \
                                           'has_churned', 'churn_date', 'churn_epoch'])

### asOfDate

In [389]:
user_first_deposit_date

datetime.date(2021, 12, 28)

In [390]:
dates[dates['date'].apply(lambda x: x>=user_first_deposit_date)].sort_values('date').iloc[:,0]

6     2021-12-28
3     2021-12-29
5     2021-12-30
4     2021-12-31
9     2022-01-01
         ...    
98    2022-03-28
94    2022-03-29
92    2022-03-30
93    2022-03-31
95    2022-04-01
Name: date, Length: 95, dtype: dbdate

In [391]:
user_deposit_table['as_of_date'] = dates[dates['date'].apply(lambda x: x>=user_first_deposit_date)].sort_values('date').iloc[:,0]

### userAddress

In [392]:
user_deposit_table['user_address'] = current_user_address

### totalValueDepositedUSD
TODO: Add logic to covert to USD

#### inner loop logic

In [393]:
user_deposit_table.shape[0]

95

In [394]:
user_deposit_table.index = range(user_deposit_table.shape[0])

In [395]:
query_date = user_deposit_table['as_of_date'][0].isoformat()
pd.read_gbq(query=f"select sum(deposit_initiated_amt) as total_deposited from transactions.fact_deposits where user_address=\'{current_user_address}\' and date(deposit_initiated_ts) <= date(\'{query_date}\')")

,total_deposited
0,0.00456


In [396]:
# need to add market marking code
user_deposit_table['total_deposited_USD'] = [pd.read_gbq(query=f"select sum(deposit_initiated_amt) as total_deposited from transactions.fact_deposits where user_address=\'{current_user_address}\' and date(deposit_initiated_ts) <= date(\'{query_date}\')").iloc[0,0] for query_date in user_deposit_table['as_of_date'].tolist()]

### totalValueWithdrawnUSD
TODO: Add logic to convert to USD

In [397]:
query_date = user_deposit_table['as_of_date'][0].isoformat()
pd.read_gbq(query=f"select sum(withdrawal_initiated_amt) as total_withdrawn from transactions.fact_withdrawals where user_address=\'{current_user_address}\' and date(withdrawal_initiated_ts) <= date(\'{query_date}\')").fillna(0).iloc[0,0]

0.0

In [398]:
# need to add market marking code
# will need to change to the usd amount from the trans tables in prod
user_deposit_table['total_withdrawn_USD'] = [pd.read_gbq(query=f"select sum(withdrawal_initiated_amt) as total_withdrawn from transactions.fact_withdrawals where user_address=\'{current_user_address}\' and date(withdrawal_initiated_ts) <= date(\'{query_date}\')").fillna(0).iloc[0,0] for query_date in user_deposit_table['as_of_date'].tolist()]

### totalValueLockedUSD

In [399]:
user_deposit_table['total_value_locked_USD'] = user_deposit_table['total_deposited_USD'] - user_deposit_table['total_withdrawn_USD']

### firstDepositDate

In [400]:
user_deposit_table['first_deposit_date'] =  user_first_deposit_date

### firstDepositEpoch

In [401]:
user_deposit_table['first_deposit_epoch'] = pd.read_gbq(query=f'select deposit_initiated_epoch from transactions.fact_deposits where date(deposit_initiated_ts) = \'{user_first_deposit_date}\'')

### firstDepositAmount

In [402]:
user_deposit_table['first_deposit_amount'] = pd.read_gbq(query=f'select sum(deposit_initiated_amt) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{user_first_deposit_date.isoformat()}\'').iloc[0,0]

### firstDepositToken

In [403]:
user_deposit_table['first_deposit_token'] = pd.read_gbq(query=f'select deposited_asset from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{user_first_deposit_date.isoformat()}\'').iloc[0,0]

### tvlDelta30Days

In [404]:
current_as_of_date = user_deposit_table['as_of_date'].iloc[3]

# we can also perform this logic with GBQ SQL
#      DATE_ADD(DATE "2022-01-19", INTERVAL 30 DAY)
window_end_date = current_as_of_date - datetime.timedelta(30)

q = f'\
    with window_deposits as \
         (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
         from transactions.fact_deposits \
         where user_address = \'{current_user_address}\' \
         and date(deposit_initiated_ts) between \'{window_end_date.isoformat()}\' and \'{current_as_of_date.isoformat()}\'\
         group by user_address) \
    select * from window_deposits'

r = pd.read_gbq(query=q)
r

,user_address,deposits
0,6o2ASdEoRGDZaaHRupBmocL7wKWNpyMXkFKePGsSs79p,0.009967


In [405]:
# tvl_list = []
# for table_date in user_deposit_table['as_of_date']:
#     current_as_of_date = table_date
#     # we can also perform this logic with GBQ SQL
#     #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
#     window_end_date = current_as_of_date - datetime.timedelta(30)
#     window_delta = pd.read_gbq(query=f'\
#         with window_deposits as \
#          (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
#          from transactions.fact_deposits \
#          where user_address = \'{current_user_address}\' \
#          and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#              group by user_address) \
#         , window_withdrawals as \
#             (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
#             from transactions.fact_withdrawals \
#             where user_address = \'{current_user_address}\' \
#             and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#             group by user_address) \
#         select \
#              IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
#         from window_deposits wd\
#         left join window_withdrawals ww on wd.user_address = ww.user_address \
#         ')
#     if window_delta.empty:
#         # print(window_end_date, current_as_of_date)
#         # print(table_date, window_delta.empty)
#         tvl_list.append(0.0)
#     else:
#         # print(window_end_date, current_as_of_date)
#         # print(window_delta)
#         # print(table_date, window_delta.iloc[0,0])
#         tvl_list.append(window_delta.iloc[0,0])
    
# user_deposit_table['tvl_delta_30_days'] = tvl_list

### tvlDelta60Days

In [406]:
# tvl_list = []
# for table_date in user_deposit_table['as_of_date']:
#     current_as_of_date = table_date
#     # we can also perform this logic with GBQ SQL
#     #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
#     window_end_date = current_as_of_date - datetime.timedelta(60)
#     window_delta = pd.read_gbq(query=f'\
#         with window_deposits as \
#          (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
#          from transactions.fact_deposits \
#          where user_address = \'{current_user_address}\' \
#          and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#              group by user_address) \
#         , window_withdrawals as \
#             (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
#             from transactions.fact_withdrawals \
#             where user_address = \'{current_user_address}\' \
#             and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#             group by user_address) \
#         select \
#              IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
#         from window_deposits wd\
#         left join window_withdrawals ww on wd.user_address = ww.user_address \
#         ')
#     if window_delta.empty:
#         # print(window_end_date, current_as_of_date)
#         # print(table_date, window_delta.empty)
#         tvl_list.append(0.0)
#     else:
#         # print(window_end_date, current_as_of_date)
#         # print(window_delta)
#         # print(table_date, window_delta.iloc[0,0])
#         tvl_list.append(window_delta.iloc[0,0])

# user_deposit_table['tvl_delta_60_days'] = tvl_list

### tvlDelta90Days

In [407]:
# tvl_list = []
# for table_date in user_deposit_table['as_of_date']:
#     current_as_of_date = table_date
#     # we can also perform this logic with GBQ SQL
#     #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
#     window_end_date = current_as_of_date - datetime.timedelta(90)
#     window_delta = pd.read_gbq(query=f'\
#         with window_deposits as \
#          (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
#          from transactions.fact_deposits \
#          where user_address = \'{current_user_address}\' \
#          and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#              group by user_address) \
#         , window_withdrawals as \
#             (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
#             from transactions.fact_withdrawals \
#             where user_address = \'{current_user_address}\' \
#             and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
#             group by user_address) \
#         select \
#              IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
#         from window_deposits wd\
#         left join window_withdrawals ww on wd.user_address = ww.user_address \
#         ')
#     if window_delta.empty:
#         # print(window_end_date, current_as_of_date)
#         # print(table_date, window_delta.empty)
#         tvl_list.append(0.0)
#     else:
#         # print(window_end_date, current_as_of_date)
#         # print(window_delta)
#         # print(table_date, window_delta.iloc[0,0])
#         tvl_list.append(window_delta.iloc[0,0])
    
# user_deposit_table['tvl_delta_90_days'] = tvl_list

### tvlDelta1Epoch

### tvlDelta2Epoch

### tvlDelta3Epoch

### tvlDelta4Epoch

### tvlDelta5Epoch

### lastDepositDate

In [408]:
pd.read_gbq(query=f'select max(date(deposit_initiated_ts)) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) <= \'{current_as_of_date}\'')

,f0_
0,2021-12-30


In [409]:
last_deposit_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max(date(deposit_initiated_ts)) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) <= \'{current_as_of_date}\'')
    last_deposit_list.append(r.iloc[0,0])
user_deposit_table['last_deposit_date'] = last_deposit_list

### lastDepositEpoch

In [410]:
last_deposit_epoch_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max((deposit_initiated_epoch)) from transactions.fact_deposits where date(deposit_initiated_ts) <= \'{current_as_of_date}\'')
    last_deposit_epoch_list.append(r.iloc[0,0])
user_deposit_table['last_deposit_epoch'] = last_deposit_epoch_list

### lastDepositAmount & lastDepositToken

In [411]:
last_deposit_amount_list = []
last_deposit_token_list = []

for table_date in user_deposit_table['last_deposit_date']:
    r = pd.read_gbq(query=f'select deposited_asset, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{table_date}\'')
    
    last_deposit_token_list.append(r.iloc[0,0])
    last_deposit_amount_list.append(r.iloc[0,1])
user_deposit_table['last_deposit_token'] = last_deposit_token_list
user_deposit_table['last_deposit_amount'] = last_deposit_amount_list

In [426]:
current_user_address

'6o2ASdEoRGDZaaHRupBmocL7wKWNpyMXkFKePGsSs79p'

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [431]:
%%timeit -n 10  
table_date = '2021-12-28' 
query = f'select deposited_asset, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{table_date}\''
query_job = client.query(query)  # Make an API request.
query_job.to_dataframe()

1.36 s ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [430]:
%%timeit -n 10 
table_date = '2021-12-28' 
query = f'select deposited_asset, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{table_date}\''

pd.read_gbq(query=query)

1.49 s ± 688 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### daysSinceLastDeposit

In [412]:
days_since_last_deposit_list = []
for as_of_date, last_deposit_date in user_deposit_table[['as_of_date', 'last_deposit_date']].to_records(index=False).tolist():
    days_since_last_deposit_list.append((as_of_date - last_deposit_date).days)
user_deposit_table['days_since_last_deposit'] = days_since_last_deposit_list

### epochsSinceLastDeposit

### firstWithdrawalDate

In [413]:
r = pd.read_gbq(query=f'select min(date(withdrawal_initiated_ts)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\'')
if r.empty:
    pass
else:
    user_deposit_table['first_withdrawal_date'] = r.iloc[0,0]

### firstWithdrawalEpoch

In [414]:
first_withdrawal_date = user_deposit_table['first_withdrawal_date'].iloc[0]
r = pd.read_gbq(query=f'select (withdrawal_initiated_epoch) from transactions.fact_withdrawals where date(withdrawal_initiated_ts) <= \'{first_withdrawal_date}\'')
user_deposit_table['first_withdrawal_epoch'] = r.iloc[0,0]

### firstWithdrawalAmount & firstWithdrawalToken

In [415]:
first_withdrawal_date = user_deposit_table['first_withdrawal_date'].iloc[0]
r = pd.read_gbq(query=f'select deposited_asset, withdrawal_initiated_amt from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{first_withdrawal_date}\'')
r
user_deposit_table['first_withdrawal_token'] = r.iloc[0,0]
user_deposit_table['first_withdrawal_amount'] = r.iloc[0,1]

### lastWithdrawalDate

In [416]:
last_withdrawal_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max(date(withdrawal_initiated_ts)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) <= \'{current_as_of_date}\'')
    last_withdrawal_list.append(r.iloc[0,0])
user_deposit_table['last_withdrawal_date'] = last_withdrawal_list

### lastWithdrawalEpoch

In [417]:
if pd.isnull(user_deposit_table['last_withdrawal_date'].iloc[15]):
    print('its true')
else:
    print('its false')

its true


In [418]:
last_withdrawal_epoch_list = []

for table_date in user_deposit_table['last_withdrawal_date']:
    if pd.isnull(table_date):
        last_withdrawal_epoch_list.append(None)
    else:
        r = pd.read_gbq(query=f'select max((withdrawal_initiated_epoch)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{table_date}\'')
        last_withdrawal_epoch_list.append(r.iloc[0,0])
user_deposit_table['last_withdrawal_epoch'] = last_withdrawal_epoch_list

### lastWithdrawalAmount & lastWithdrawalToken

In [419]:
last_withdrawal_amount_list = []
last_withdrawal_token_list = []

for table_date in user_deposit_table['last_withdrawal_date']:
    current_as_of_date = table_date
    # print(current_as_of_date)
    if pd.isnull(current_as_of_date):
        last_withdrawal_token_list.append(None)
        last_withdrawal_amount_list.append(None)
    else:
        r = pd.read_gbq(query=f'select deposited_asset, withdrawal_initiated_amt from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{current_as_of_date}\'')

        last_withdrawal_token_list.append(r.iloc[0,0])
        last_withdrawal_amount_list.append(r.iloc[0,1])
user_deposit_table['last_withdrawal_token'] = last_withdrawal_token_list
user_deposit_table['last_withdrawal_amount'] = last_withdrawal_amount_list

### hasChurned

In [420]:
user_deposit_table['has_churned'] = (user_deposit_table['total_deposited_USD'] - user_deposit_table['total_withdrawn_USD']).apply(lambda x: x <= 0)

### churnDate

In [421]:
churn_state = user_deposit_table['has_churned'].tolist()
if any(churn_state):
    user_deposit_table['churn_date'] = None
else:
    churn_date_list = []

    for index in range(len(churn_state)-1):
        if index == 0:
            churn_date_list.append(None)
        if (churn_state[index-1] == False ) & (churn_state[index] == True):
            churn_date_list.append(user_deposit_table['as_of_date'].iloc[index])
        else:
            churn_date_list.append(churn_date_list[index-1])

    user_deposit_table['churn_date'] = churn_date_list

### churnEpoch

# Table Peeks

In [422]:
pd.read_gbq(query=f'select deposit_initiated_ts, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\'')

,deposit_initiated_ts,deposit_initiated_amt
0,2021-12-30 13:19:40+00:00,0.005407
1,2021-12-28 15:04:04+00:00,0.004560
2,2022-01-14 18:53:07+00:00,66.852187
3,2022-01-04 21:58:02+00:00,5.000000
4,2022-01-03 18:46:15+00:00,250.000000


In [423]:
user_deposit_table.iloc[:5,:][['as_of_date', 'last_deposit_date']]

,as_of_date,last_deposit_date
0,2021-12-28,2021-12-28
1,2021-12-29,2021-12-28
2,2021-12-30,2021-12-30
3,2021-12-31,2021-12-30
4,2022-01-01,2021-12-30


In [424]:
user_deposit_table['as_of_date'].min().isoformat()

'2021-12-28'

In [425]:
user_deposit_table.iloc[17,:]

user_address                 6o2ASdEoRGDZaaHRupBmocL7wKWNpyMXkFKePGsSs79p
as_of_date                                                     2022-01-14
total_value_locked_USD                                          71.862155
total_deposited_USD                                            321.862154
total_withdrawn_USD                                            249.999999
tvl_delta_30_days                                                     NaN
tvl_delta_60_days                                                     NaN
tvl_delta_90_days                                                     NaN
tvl_delta_1_epoch                                                     NaN
tvl_delta_2_epoch                                                     NaN
tvl_delta_3_epoch                                                     NaN
tvl_delta_4_epoch                                                     NaN
tvl_delta5_epoch                                                      NaN
first_deposit_date                    